# CH 6 - Data Loading, Storage, and File Formats

## 6.1 Reading and Writing Data in Text Format

In [1]:
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [4]:
import pandas as pd 

ex = lambda x: 'examples/' + x

In [7]:
df = pd.read_csv(ex('ex1.csv'))
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
pd.read_table(ex('ex1.csv'), sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
!cat examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [10]:
# You can allow pandas to assign default column names, or you can specify names yourself:
pd.read_csv(ex('ex2.csv'), names=['a','b','c','d','message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [11]:
# Suppose you wanted the message column to be the index of the returned DataFrame.
names = ['a','b','c','d','message']
pd.read_csv(ex('ex2.csv'), names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [12]:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [13]:
pd.read_csv(ex('csv_mindex.csv'), index_col=['key1','key2'])

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [15]:
# In some cases, a table might not have a fixed delimiter, 
# using whitespace or some other pattern to separate fields.
list(open(ex('ex3.txt')))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [16]:
pd.read_table(ex('ex3.txt'), sep='\s+')

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [17]:
pd.read_csv(ex('ex3.txt'), sep='\s+')

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [18]:
# You can skip the first, third, and fourth rows of a file with skiprows
!cat examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [19]:
pd.read_csv(ex('ex4.csv'), skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [20]:
# By default, pandas uses a set of commonly occurring 
# sentinels, such as NA and NULL:
!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [21]:
pd.read_csv(ex('ex5.csv'))

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [22]:
pd.read_csv(ex('ex5.csv'), na_values=['NULL'])

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [23]:
# Different NA sentinels can be specified for each column in a dict
sentinels = {'message': ['foo','NA'], 'something': ['two']}

pd.read_csv(ex('ex5.csv'), na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Reading Text Files in Pieces

In [24]:
# Before we look at a large file, we make the pandas display settings more compact
pd.options.display.max_rows = 10

In [25]:
pd.read_csv(ex('ex6.csv'))

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [26]:
# If you want to only read a small number of rows 
# (avoiding reading the entire file), specify that with nrows
pd.read_csv(ex('ex6.csv'), nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [41]:
# To read a file in pieces, specify a chunksize as a number of rows:
chunker = pd.read_csv(ex('ex6.csv'), chunksize=500)
chunker

In [42]:
# The TextParser object returned by read_csv allows you to iterate over the parts of
# the file according to the chunksize . For example, we can iterate over ex6.csv , aggre‐
# gating the value counts in the 'key' column like so:

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)
tot[:10]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### Writing Data to Text Format

In [44]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [46]:
# writing to sys.stdout so it prints the text result to the console
import sys
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [47]:
data.to_csv(sys.stdout, index=False, columns=['a','b','c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


### JSON Data

In [49]:
obj = """
{"name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
{"name": "Katie", "age": 38,
"pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [50]:
import json

# To convert a JSON string to Python form, use json.loads
result = json.loads(obj)

result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [52]:
# json.dumps, on the other hand, converts a Python object back to JSON
asjson = json.dumps(result)

asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

In [53]:
# How you convert a JSON object or list of objects to a DataFrame or some
# other data structure for analysis will be up to you.

siblings = pd.DataFrame(result['siblings'], columns=['name','age'])
siblings

,name,age
0,Scott,30
1,Katie,38


In [54]:
# The pandas.read_json can automatically convert JSON datasets in 
# specific arrangements into a Series or DataFrame.
!cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [56]:
# The default options for pandas.read_json assume that each object in 
# the JSON array is a row in the table
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [58]:
# If you need to export data from pandas to JSON, one way is to use
# the to_json methods on Series and DataFrame

print(data.to_json(), '\n')
print(data.to_json(orient='records'))

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}} 

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


### XML and HTML: Web Scraping

In [59]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')

In [60]:
len(tables)

1

In [61]:
failures = tables[0]

failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [64]:
close_timestamps = pd.to_datetime(failures['Closing Date'])

In [67]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

In [72]:
from lxml import objectify

path = 'examples/Performance_MNR.xml'
parsed = objectify.parse(open(path))

# get root node
root = parsed.getroot()

In [76]:
# root.INDICATOR returns a generator yielding each <INDICATOR> XML element. For
# each record, we can populate a dict of tag names (like YTD_ACTUAL ) to data values

data = []

skip_fields =['PARENT_SEQ', 'INDICATOR_SEQ',
'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [77]:
el_data

{'AGENCY_NAME': 'Metro-North Railroad',
 'INDICATOR_NAME': 'Escalator Availability',
 'DESCRIPTION': 'Percent of the time that escalators are operational systemwide. The availability rate is based on physical observations performed the morning of regular business days only. This is a new indicator the agency began reporting in 2009.',
 'PERIOD_YEAR': 2011,
 'PERIOD_MONTH': 12,
 'CATEGORY': 'Service Indicators',
 'FREQUENCY': 'M',
 'INDICATOR_UNIT': '%',
 'YTD_TARGET': 97.0,
 'YTD_ACTUAL': '',
 'MONTHLY_TARGET': 97.0,
 'MONTHLY_ACTUAL': ''}

In [78]:
perf = pd.DataFrame(data)

perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95,96.9,95,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95,96,95,95
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95,96.3,95,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95,96.8,95,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95,96.6,95,95.8


## 6.2 Binary Data Formats

In [79]:
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [80]:
frame.to_pickle('examples/frame_pickle')

In [82]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* pickle is only recommended as a short-term storage format. The
problem is that it is hard to guarantee that the format will be stable
over time; an object pickled today may not unpickle with a later
version of a library. We have tried to maintain backward compati‐
bility when possible, but at some point in the future it may be nec‐
essary to “break” the pickle format.

### Reading Microsoft Excel Files

In [2]:
import pandas as pd
xlsx = pd.ExcelFile('examples/ex1.xlsx')

In [3]:
xlsx

In [4]:
pd.read_excel(xlsx, "Sheet1")

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


* To write pandas data to Excel format, you must first create an `ExcelWriter` , then
write data to it using pandas objects’ `to_excel` method:

In [5]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')

In [9]:
writer = pd.ExcelWriter('examples/ex2.xlsx')

frame.to_excel('examples/ex2.xlsx')

## 6.3 Interacting with Web APIs

In [10]:
import requests

url = 'https://api.github.com/repos/pandas-dev/pandas/issues'

resp = requests.get(url)

resp

<Response [200]>

In [13]:
data = resp.json()

data[0]['title']

'DOC/API: Discussion on whether __finalize__ is public'

In [14]:
# We can pass data directly to DataFrame and extract fields of interest:

issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])

In [16]:
issues.head()

,number,title,labels,state
0,33338,DOC/API: Discussion on whether __finalize__ is...,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
1,33337,DOC/PLT: Add `stacked` in doc and doc example ...,[],open
2,33336,PLT: Order of plots does not preserve the colu...,[],open
3,33334,BLD: Increase minimum version of Cython to 0.2...,"[{'id': 527603109, 'node_id': 'MDU6TGFiZWw1Mjc...",open
4,33333,DEPR: deprecate _data when getting BlockManager,"[{'id': 87485152, 'node_id': 'MDU6TGFiZWw4NzQ4...",open


## 6.4 Interacting with Databases

In [20]:
# I’ll create a SQLite database using Python’s built-in sqlite3 driver:
import sqlite3

query = '''
create table test
(a varchar(20), b varchar(20), c real, d integer);
'''

conn = sqlite3.connect('/home/jovyan/work/db/mydata.sqlite')

conn.execute(query)

conn.commit()

In [21]:
# insert a few rows of data:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

conn.executemany('insert into test values(?,?,?,?)', data)

conn.commit()

In [22]:
cursor = conn.execute('select * from test')

rows = cursor.fetchall()

rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [23]:
# You can pass the list of tuples to the DataFrame constructor, 
# but you also need the column names, contained in the 
# cursor’s description attribute:

cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [24]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


* This is quite a bit of munging that you’d rather not repeat each time you query the
database. The SQLAlchemy project is a popular Python SQL toolkit that abstracts
away many of the common differences between SQL databases. `pandas` has a
`read_sql` function that enables you to read data easily from a general SQLAlchemy
connection.

In [29]:
import sqlalchemy as sqla

db = sqla.create_engine('sqlite:///' + '/home/jovyan/work/db/mydata.sqlite')

In [30]:
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
